In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import boto3
import time
from time import sleep
from lxml import html
import os

### 定数

In [2]:
REGION_NAME = 'ap-northeast-1'

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")

DATASET_GROUP_NAME = "word_suggest_recommendation"
INTERACTIONS_FILENAME = "search_log.csv" # 要修正
ROLE_NAME = "PersonalizeRoleAmazonPantryAwsSimilarItems"

# レシピ名
# SIMS_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-sims"
# SIMS_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together"
# SIMS_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed"
SIMS_RECIPE_ARN = "arn:aws:personalize:::recipe/aws-similar-items"

SCHEMA_NAME = "suggest-word-1"

### データ読み込み

In [3]:
search_log_df = pd.read_csv(INTERACTIONS_FILENAME)

search_log_df.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE
0,user_001,coffee,1704067200,purchase
1,user_001,tea,1704067200,purchase
2,user_001,milk,1704067200,purchase
3,user_002,bread,1704153600,purchase
4,user_002,cheese,1704153600,purchase


In [4]:
search_log_df.describe(include='all')

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE
count,4913,4913,4.913000e+03,4913
unique,30,85,NaN,1
top,user_013,cereal,NaN,purchase
freq,213,80,NaN,4913
mean,NaN,NaN,1.719666e+09,NaN
std,NaN,NaN,9.062495e+06,NaN
min,NaN,NaN,1.704046e+09,NaN
25%,NaN,NaN,1.711964e+09,NaN
50%,NaN,NaN,1.719433e+09,NaN
75%,NaN,NaN,1.727557e+09,NaN


### データセットグループの作成

In [5]:
personalize = boto3.client(
    'personalize',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

personalize_runtime = boto3.client(
    'personalize-runtime', 
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

create_dataset_group_response = personalize.create_dataset_group(
    name = DATASET_GROUP_NAME,
    # domain='ECOMMERCE',
)

dataset_group = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-northeast-1:627614495110:dataset-group/word_suggest_recommendation",
  "ResponseMetadata": {
    "RequestId": "7605c331-94b6-4185-84d5-c5c7177a0687",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 01 Jun 2025 07:03:25 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "111",
      "connection": "keep-alive",
      "x-amzn-requestid": "7605c331-94b6-4185-84d5-c5c7177a0687",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### max_time を上限として、データセットグループの作成が完了したことを確認し続ける

In [6]:
max_time = time.time() + 3*60*60 # 3 hours

while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    if status == "ACTIVE":
        print(f"Dataset group create succeeded for {dataset_group}")
        break
    elif status == "CREATE FAILED":
        print(f"Create failed for {dataset_group}")
        break
                
    time.sleep(10)

Dataset group create succeeded for arn:aws:personalize:ap-northeast-1:627614495110:dataset-group/word_suggest_recommendation


### インタラクションデータセットのスキーマとデータセットを作成する

In [7]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}

### すでに Schema を作っていたら削除、なければ作成

In [8]:
# スキーマの一覧を取得
response = personalize.list_schemas()
schema_arn = None

for schema in response['schemas']:
    if schema['name'] == SCHEMA_NAME:
        schema_arn = schema['schemaArn']
        break

if schema_arn:
    personalize.delete_schema(
        schemaArn=schema_arn
    )
    
    sleep(5)
        
    create_schema_response = personalize.create_schema(
        name = SCHEMA_NAME,
        # domain = "ECOMMERCE",
        schema = json.dumps(interactions_schema),
    )

else:
    create_schema_response = personalize.create_schema(
        name = SCHEMA_NAME,
        # domain = "ECOMMERCE",
        schema = json.dumps(interactions_schema),
    )

In [9]:
# スキーマの中身を確認
interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-northeast-1:627614495110:schema/suggest-word-1",
  "ResponseMetadata": {
    "RequestId": "3f3854e4-8fd7-4445-a6e0-99bdd28107f8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 01 Jun 2025 07:03:51 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "85",
      "connection": "keep-alive",
      "x-amzn-requestid": "3f3854e4-8fd7-4445-a6e0-99bdd28107f8",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### 先ほど作成したスキーマを指定して、データセットグループにInteractionsデータセットを作成する

In [10]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "amazon-pantry-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group,
    schemaArn = interaction_schema_arn
)

interactions_dataset = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-northeast-1:627614495110:dataset/word_suggest_recommendation/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "d3a5ec61-6b70-430c-aed5-be07afbc4383",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 01 Jun 2025 07:03:51 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "d3a5ec61-6b70-430c-aed5-be07afbc4383",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [11]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_response = personalize.describe_dataset(
        datasetArn = interactions_dataset
    )
    status = describe_dataset_response["dataset"]['status']
    if status == "ACTIVE":
        print(f"Dataset import succeeded for {interactions_dataset}")
        break
    elif status == "CREATE FAILED":
        print(f"Create failed for {interactions_dataset}")
        break
                
    time.sleep(60)

Dataset import succeeded for arn:aws:personalize:ap-northeast-1:627614495110:dataset/word_suggest_recommendation/INTERACTIONS
CPU times: user 34.1 ms, sys: 4.77 ms, total: 38.8 ms
Wall time: 1min


### S3でインタラクションCSVをステージングする
### 先ほど作成したInteractions CSVを先ほど作成したPersonalizeデータセットにアップロードする前に、S3バケットにCSVをステージングする必要がある
### S3バケットを作成し、Interactions CSVファイルをバケットにコピーしよう。

In [12]:
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,    
)

ACCOUNT_ID = boto3.client(
    'sts',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
).get_caller_identity().get('Account')

BUCKET_NAME = f"amazon-pantry-personalize-example-{ACCOUNT_ID}"

print(BUCKET_NAME)

if REGION_NAME == "us-east-1":
    s3.create_bucket(Bucket=BUCKET_NAME)
else:
    s3.create_bucket(
        Bucket=BUCKET_NAME,
        CreateBucketConfiguration={'LocationConstraint': REGION_NAME}
    )

amazon-pantry-personalize-example-627614495110


### S3 へのアップロード

In [13]:
boto3.Session().resource(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
).Bucket(BUCKET_NAME).Object(
    INTERACTIONS_FILENAME).upload_file(
    INTERACTIONS_FILENAME)

### S3バケットポリシーとIAMロールの作成
### データセットのインポートジョブをPersonalizeに送信する前に、Personalizeにバケットへのアクセス権を与えるバケットポリシーとIAMロールを作成する必要があります。

In [14]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{BUCKET_NAME}",
                f"arn:aws:s3:::{BUCKET_NAME}/*",
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=BUCKET_NAME, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '3M95FE3T0REZ3YV4',
  'HostId': 'TfIkJRFfm1TJYMXd1TdhaX/SfHDEBFl0bT2jdid8YWvhNjVEHJXB80UKTBI/ps+b7WvkL8WuVPc=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'TfIkJRFfm1TJYMXd1TdhaX/SfHDEBFl0bT2jdid8YWvhNjVEHJXB80UKTBI/ps+b7WvkL8WuVPc=',
   'x-amz-request-id': '3M95FE3T0REZ3YV4',
   'date': 'Sun, 01 Jun 2025 07:04:57 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [15]:
iam = boto3.client(
    "iam",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME,
)

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam.create_role(
        RoleName = ROLE_NAME,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    )
except:
    create_role_response = iam.get_role(RoleName=ROLE_NAME)

In [16]:
role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::627614495110:role/PersonalizeRoleAmazonPantryAwsSimilarItems


### インタラクションデータセットのインポート
### S3バケットにあるステージングされたInteractions CSVを、作成したPersonalizeデータセットにインポートする準備を行う

In [17]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "amazon-pantry-interactions-import",
    datasetArn = interactions_dataset,
    dataSource = {
        "dataLocation": f"s3://{BUCKET_NAME}/{INTERACTIONS_FILENAME}"
    },
    roleArn = role_arn
)

dataset_import_job = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-northeast-1:627614495110:dataset-import-job/amazon-pantry-interactions-import",
  "ResponseMetadata": {
    "RequestId": "ab439e66-9a9d-4ea2-b4cf-ea8a45405db0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 01 Jun 2025 07:04:58 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "126",
      "connection": "keep-alive",
      "x-amzn-requestid": "ab439e66-9a9d-4ea2-b4cf-ea8a45405db0",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### 相互作用データセットのインポートジョブの完了を待つ
### 次のセルは、両方のインポート・ジョブが完了するまで待機します。

In [18]:
%%time

in_progress_import_arns = [ dataset_import_job ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    if status == "ACTIVE":
        print(f"Dataset import succeeded for {dataset_import_job}")
        break
    elif status == "CREATE FAILED":
        print(f"Create failed for {dataset_import_job}")
        break
                
    time.sleep(60)

Dataset import succeeded for arn:aws:personalize:ap-northeast-1:627614495110:dataset-import-job/amazon-pantry-interactions-import
CPU times: user 188 ms, sys: 26.1 ms, total: 214 ms
Wall time: 4min 3s


### ソリューションとソリューションバージョンの作成
### 各データセットグループのデータのユーザパーソナライズのレシピを使って、ソリューションとソリューションのバージョンを作成します。
### まず、利用可能なパーソナライズレシピを列挙します。

In [19]:
response = personalize.list_recipes(
    # domain='ECOMMERCE'
)

for recipe in response['recipes']:
    print(f"Name: {recipe['name']}, ARN: {recipe['recipeArn']}")

Name: aws-ecomm-customers-who-viewed-x-also-viewed, ARN: arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed
Name: aws-ecomm-frequently-bought-together, ARN: arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together
Name: aws-ecomm-popular-items-by-purchases, ARN: arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases
Name: aws-ecomm-popular-items-by-views, ARN: arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-views
Name: aws-ecomm-recommended-for-you, ARN: arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you
Name: aws-item-affinity, ARN: arn:aws:personalize:::recipe/aws-item-affinity
Name: aws-item-attribute-affinity, ARN: arn:aws:personalize:::recipe/aws-item-attribute-affinity
Name: aws-next-best-action, ARN: arn:aws:personalize:::recipe/aws-next-best-action
Name: aws-personalized-ranking, ARN: arn:aws:personalize:::recipe/aws-personalized-ranking
Name: aws-personalized-ranking-v2, ARN: arn:aws:personalize:::recipe/aws-personaliz

### レシピの選択

### まず、各レシピのソリューションとソリューション・バージョンを作成する。

In [21]:
sims_create_solution_response = personalize.create_solution(
    name = "amazon-pantry-sims-solution-example",
    datasetGroupArn = dataset_group,
    recipeArn = SIMS_RECIPE_ARN
)

sims_solution_arn = sims_create_solution_response['solutionArn']

In [22]:
sims_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [23]:
sims_solution_version_arn = sims_solution_version_response['solutionVersionArn']
print(json.dumps(sims_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example/bfcea417",
  "ResponseMetadata": {
    "RequestId": "1c98948b-9664-422c-a8c4-a7b650b0a0a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 01 Jun 2025 07:10:38 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "126",
      "connection": "keep-alive",
      "x-amzn-requestid": "1c98948b-9664-422c-a8c4-a7b650b0a0a4",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### ソリューション・バージョンがアクティブになるのを待つ
### 最後に、ソリューションバージョンが作成され終わるのを待ちます。このステップでPersonalizeはデータセットと選択されたレシピに基づいて機械学習モデルをトレーニングします。
### また、Personalizeはインタラクションのデータセットをトレーニング部分と評価部分に分割し、トレーニング済みのモデルに対するレコメンデーションの品質を評価できるようにします。
### 説明データを含むデータセットグループのソリューションバージョンは、説明データを含まないものよりもトレーニングに時間がかかることがわかります

In [24]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    version_response = personalize.describe_solution_version(
        solutionVersionArn = sims_solution_version_arn
    )
    status = version_response["solutionVersion"]["status"]
    
    if status == "ACTIVE":
        print(f"Build succeeded for {sims_solution_version_arn}")
        break
    elif status == "CREATE FAILED":
        print(f"Build failed for {sims_solution_version_arn}")
        break
        
    time.sleep(60)

Build succeeded for arn:aws:personalize:ap-northeast-1:627614495110:solution/amazon-pantry-sims-solution-example/bfcea417
CPU times: user 644 ms, sys: 64.8 ms, total: 709 ms
Wall time: 14min 6s


### 処理時間

In [25]:
response = personalize.describe_solution_version(solutionVersionArn = sims_solution_version_arn)
training_hours_sims = response['solutionVersion']['trainingHours']
print(training_hours_sims)

0.732


### キャンペーンの作成

In [26]:
sims_solution = {
        "solution_arn": sims_solution_arn,
        "solution_version_arn": sims_solution_version_arn
}

In [27]:
create_campaign_response = personalize.create_campaign(
    name = f"personalize-demo-sims-example",
    solutionVersionArn = sims_solution['solution_version_arn'],
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:' + campaign_arn)

campaign_arn:arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example


In [28]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print(f"Campaign: {campaign_arn} {status}")

    if status == "ACTIVE" or status == "CREATE FAILED":
        break

    time.sleep(60)

Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE PENDING
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personalize-demo-sims-example CREATE IN_PROGRESS
Campaign: arn:aws:personalize:ap-northeast-1:627614495110:campaign/personali

### 推薦実行

In [29]:
search_log_df.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE
0,user_001,coffee,1704067200,purchase
1,user_001,tea,1704067200,purchase
2,user_001,milk,1704067200,purchase
3,user_002,bread,1704153600,purchase
4,user_002,cheese,1704153600,purchase


In [30]:
search_log_count_df = pd.DataFrame(
    {
        "ITEM_ID": search_log_df['ITEM_ID'].value_counts().index,
        "COUNT": search_log_df['ITEM_ID'].value_counts().values,
    }
)

search_log_count_df.head()

,ITEM_ID,COUNT
0,cereal,80
1,honey,75
2,mushroom,70
3,fork,70
4,plate,70


In [31]:
search_query = search_log_count_df.sample()["ITEM_ID"].item()
print(search_query)

lemon


In [32]:
search_log_count_df.query(f'ITEM_ID=="{search_query}"')

,ITEM_ID,COUNT
9,lemon,66


In [33]:
# 与えるデータ
recommended_item_df = search_log_count_df.query(f'ITEM_ID=="{search_query}"')
recommended_item_df

,ITEM_ID,COUNT
9,lemon,66


In [34]:
### search_query のワードが使われているTIMESTAMP
cleaner_timestamps = search_log_df.loc[search_log_df['ITEM_ID'] == search_query, 'TIMESTAMP'].unique()
cleaner_timestamps

array([1705449600, 1708387200, 1711324800, 1714176000, 1717113600,
       1719964800, 1722902400, 1725840000, 1728691200, 1731542400,
       1734480000, 1724229913, 1707486366, 1712514011, 1714190816,
       1724955612, 1707135949, 1734830850, 1718753653, 1707285648,
       1726764739, 1705472792, 1729220264, 1708938549, 1734684424,
       1734347243, 1714617726, 1724773844, 1705343792, 1719906036,
       1704429944, 1711421033, 1729862113, 1726965585, 1728654907,
       1710465951, 1716153735, 1733037489, 1723529374, 1723111910,
       1719462568, 1714300744, 1713032428, 1720058576, 1718139052,
       1734022917, 1710378458, 1718924199, 1704873899, 1730764473,
       1710183126, 1718711999, 1731721344, 1722724775, 1731825630,
       1734079700, 1734500286, 1718601306, 1713726384, 1726957925,
       1727990019, 1725820615, 1719077935, 1719593810, 1715313928,
       1710851837])

In [35]:
co_searched_df = search_log_df[search_log_df['TIMESTAMP'].isin(cleaner_timestamps)]
co_searched_word = co_searched_df.loc[search_log_df['ITEM_ID'] != search_query, 'ITEM_ID'].unique()
sorted(co_searched_word)

['avocado',
 'bag',
 'banana',
 'bean',
 'beef',
 'blueberry',
 'bottle',
 'bowl',
 'box',
 'bread',
 'broccoli',
 'burger',
 'butter',
 'cake',
 'can',
 'candy',
 'carrot',
 'cereal',
 'cheese',
 'chicken',
 'chocolate',
 'cleaner',
 'coffee',
 'container',
 'cookie',
 'corn',
 'cup',
 'detergent',
 'egg',
 'fish',
 'flour',
 'fork',
 'garlic',
 'ginger',
 'glass',
 'grape',
 'herb',
 'honey',
 'jar',
 'juice',
 'knife',
 'mango',
 'milk',
 'mint',
 'mushroom',
 'noodles',
 'onion',
 'orange',
 'pea',
 'pepper',
 'pineapple',
 'pizza',
 'plate',
 'rice',
 'salad',
 'salt',
 'sandwich',
 'shampoo',
 'soap',
 'soup',
 'spice',
 'sponge',
 'spoon',
 'strawberry',
 'sugar',
 'tissue',
 'toothpaste',
 'wine',
 'yogurt']

In [36]:
co_searched_items = co_searched_df[co_searched_df['ITEM_ID'] != search_query]
co_searched_items['ITEM_ID'].value_counts()

ITEM_ID
herb          10
strawberry     9
avocado        6
bread          5
glass          5
              ..
bowl           1
mushroom       1
jar            1
cake           1
wine           1
Name: count, Length: 69, dtype: int64

### レコメンド

In [37]:
response = personalize_runtime.get_recommendations(
    campaignArn=campaign_arn,
    itemId=search_query,
    numResults=10
)
print(response['itemList'])

[{'itemId': 'egg'}, {'itemId': 'mango'}, {'itemId': 'noodles'}, {'itemId': 'can'}, {'itemId': 'cleaner'}, {'itemId': 'sponge'}, {'itemId': 'cheese'}, {'itemId': 'mushroom'}, {'itemId': 'soap'}, {'itemId': 'spice'}]


In [38]:
# 与えたデータを元に推薦実行
recommendation_df = pd.DataFrame(
    [dic['itemId'] for dic in response['itemList']],
    columns=["ITEM_ID"]
)

recommendation_df

,ITEM_ID
0,egg
1,mango
2,noodles
3,can
4,cleaner
5,sponge
6,cheese
7,mushroom
8,soap
9,spice


### リソースの削除

In [39]:
# personalize.delete_campaign(campaignArn = campaign_arn)